# 实现MDS

In [1]:
# 实现MDS算法
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
# 生成需要降维的数据
data = np.array([[0, 2, 3, 4, 5], [2, 0, 4, 5, 6], [3, 4, 0, 7, 8], [4, 5, 7, 0, 9], [5, 6, 8, 9, 0]])
data

array([[0, 2, 3, 4, 5],
       [2, 0, 4, 5, 6],
       [3, 4, 0, 7, 8],
       [4, 5, 7, 0, 9],
       [5, 6, 8, 9, 0]])

In [9]:
# 计算距离矩阵
def cal_dist_ij(data):
    n = data.shape[0]
    dist = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist[i, j] = np.sum(np.square(data[i] - data[j]))
    return dist

dist_ij = cal_dist_ij(data)
dist_ij

array([[  0.,  11.,  40.,  73., 116.],
       [ 11.,   0.,  41.,  72., 113.],
       [ 40.,  41.,   0., 101., 140.],
       [ 73.,  72., 101.,   0., 165.],
       [116., 113., 140., 165.,   0.]])

In [7]:
# 计算dist_i
def cal_dist_i(dist_ij):
    n = dist_ij.shape[0]
    dist_i = np.zeros(n)
    for i in range(n):
        dist_i[i] = np.sum(dist_ij[i]) / n
    return dist_i

dist_i = cal_dist_i(dist_ij)
dist_i

array([ 48. ,  47.4,  64.4,  82.2, 106.8])

In [8]:
# 计算dist_j
def cal_dist_j(dist_ij):
    n = dist_ij.shape[0]
    dist_j = np.zeros(n)
    for j in range(n):
        dist_j[j] = np.sum(dist_ij[:, j]) / n
    return dist_j

dist_j = cal_dist_j(dist_ij)
dist_j

array([ 48. ,  47.4,  64.4,  82.2, 106.8])

In [10]:
# 计算dist
def cal_dist(dist_ij):
    n = dist_ij.shape[0]
    dist = np.sum(dist_ij) / (n ** 2)
    return dist

dist = cal_dist(dist_ij)
dist

69.76

In [11]:
# 计算B矩阵
def cal_B(dist_ij):
    n = dist_ij.shape[0]
    B = np.zeros((n, n))
    dist = cal_dist(dist_ij)
    dist_i = cal_dist_i(dist_ij)
    dist_j = cal_dist_j(dist_ij)
    for i in range(n):
        for j in range(n):
            B[i, j] = -0.5 * (dist_ij[i, j] - dist_i[i] - dist_j[j] + dist)
    return B

B = cal_B(dist_ij)
B

array([[ 13.12,   7.32,   1.32,  -6.28, -15.48],
       [  7.32,  12.52,   0.52,  -6.08, -14.28],
       [  1.32,   0.52,  29.52, -12.08, -19.28],
       [ -6.28,  -6.08, -12.08,  47.32, -22.88],
       [-15.48, -14.28, -19.28, -22.88,  71.92]])

In [12]:
# 计算特征值和特征向量
def cal_eig(B):
    eig_val, eig_vec = np.linalg.eig(B)
    return eig_val, eig_vec

eig_val, eig_vec = cal_eig(B)
eig_val, eig_vec

(array([ 9.11123429e+01,  5.45076416e+01,  2.32971211e+01, -4.16950939e-15,
         5.48289438e+00]),
 array([[-0.16376358, -0.22196955,  0.47514312,  0.4472136 , -0.70579744],
        [-0.14917368, -0.20472453,  0.48402142,  0.4472136 ,  0.70820784],
        [-0.21055018, -0.50243646, -0.70926975,  0.4472136 ,  0.01275352],
        [-0.35856982,  0.79977656, -0.17821555,  0.4472136 , -0.00493521],
        [ 0.88205726,  0.12935398, -0.07167924,  0.4472136 , -0.01022871]]))

In [13]:
# 选择特征值最大的k个特征向量
def select_k(eig_val, eig_vec, k):
    eig_val_index = np.argsort(-eig_val)
    eig_val = eig_val[eig_val_index]
    eig_vec = eig_vec[:, eig_val_index]
    return eig_val[:k], eig_vec[:, :k]

k = 2
eig_val_k, eig_vec_k = select_k(eig_val, eig_vec, k)
eig_val_k, eig_vec_k

(array([91.11234286, 54.50764161]),
 array([[-0.16376358, -0.22196955],
        [-0.14917368, -0.20472453],
        [-0.21055018, -0.50243646],
        [-0.35856982,  0.79977656],
        [ 0.88205726,  0.12935398]]))

In [14]:
# 计算降维后的数据
def cal_data_low(eig_val_k, eig_vec_k):
    data_low = np.dot(eig_vec_k, np.diag(np.sqrt(eig_val_k)))
    return data_low

data_low = cal_data_low(eig_val_k, eig_vec_k)
data_low

array([[-1.563169  , -1.63878542],
       [-1.42390428, -1.51146671],
       [-2.00976009, -3.70945271],
       [-3.42264886,  5.90469359],
       [ 8.41948224,  0.95501125]])

In [ ]:
# 整合代码
def MDS(X, k=2):
    def cal_dist_ij(data):
        n = data.shape[0]
        dist = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                dist[i, j] = np.sum(np.square(data[i] - data[j]))
        return dist

    def cal_dist_i(dist_ij):
        n = dist_ij.shape[0]
        dist_i = np.zeros(n)
        for i in range(n):
            dist_i[i] = np.sum(dist_ij[i]) / n
        return dist_i

    def cal_dist_j(dist_ij):
        n = dist_ij.shape[0]
        dist_j = np.zeros(n)
        for j in range(n):
            dist_j[j] = np.sum(dist_ij[:, j]) / n
        return dist_j

    def cal_dist(dist_ij):
        n = dist_ij.shape[0]
        dist = np.sum(dist_ij) / (n ** 2)
        return dist

    def cal_B(dist_ij):
        n = dist_ij.shape[0]
        B = np.zeros((n, n))
        dist = cal_dist(dist_ij)
        dist_i = cal_dist_i(dist_ij)
        dist_j = cal_dist_j(dist_ij)
        for i in range(n):
            for j in range(n):
                B[i, j] = -0.5 * (dist_ij[i, j] - dist_i[i] - dist_j[j] + dist)
        return B

    def cal_eig(B):
        eig_val, eig_vec = np.linalg.eig(B)
        return eig_val, eig_vec

    def select_k(eig_val, eig_vec, k):
        eig_val_index = np.argsort(-eig_val)
        eig_val = eig_val[eig_val_index]
        eig_vec = eig_vec[:, eig_val_index]
        return eig_val[:k], eig_vec[:, :k]

    def cal_data_low(eig_val_k, eig_vec_k):
        data_low = np.dot(eig_vec_k, np.diag(np.sqrt(eig_val_k)))
        return data_low

    dist_ij = cal_dist_ij(X)
    B = cal_B(dist_ij)
    eig_val, eig_vec = cal_eig(B)
    eig_val_k, eig_vec_k = select_k(eig_val, eig_vec, k)
    data_low = cal_data_low(eig_val_k, eig_vec_k)
    return data_low